In [2]:
from zenml import step, pipeline
from zenml.client import Client
import mlflow
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score

In [3]:

experiment_tracker = Client().active_stack.experiment_tracker

In [4]:

@step(experiment_tracker=experiment_tracker.name)
def train_model(data: pd.DataFrame) -> LinearRegression:
    """Trains a linear regression model with MLflow tracking."""
    mlflow.sklearn.autolog()
    
    X = data.drop('target', axis=1)
    y = data['target']
    X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)
    
    with mlflow.start_run():
        model = LinearRegression()
        model.fit(X_train, y_train)
    
    return model

In [5]:

@step(experiment_tracker=experiment_tracker.name)
def evaluate_model(model: LinearRegression, data: pd.DataFrame) -> None:
    """Evaluates the model and logs metrics to MLflow."""
    X = data.drop('target', axis=1)
    y = data['target']
    _, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    with mlflow.start_run():
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)